# Week 11 - Social Data Mining 2

## 1. Twitter 데이터 수집하기

트위터는 데이터를 수집할 수 있는 API를 제공한다. API는 크게 다음과 같은 두가지로 제공된다.
* REST API - 주어진 query에 맞는 데이터를 제공한다.
* Streaming API - 제시된 키워드 등이 포함된 데이터를 streaming 해준다.

트위터 API에 대응하는 여러 라이브러리가 있지만, 본 예제에서는 tweepy 를 사용한다. tweepy의 설치는 다음과 같다.
* pip install tweepy

트위터는 OAuth 인증을 통해 데이터에 접근이 가능하다. OAuth 인증을 하기 위해서는 트위터 개발자 사이트에 등록하고, 데이터 수집을 위한 앱을 등록해야 한다. 등록을 마치면 OAuth 인증을 위한 키가 제공되는데 해당 키를 이용하여 트위터 API를 사용할 수 있다.

(강의 슬라이드 참고)

### tweepy 및 OAuth 설정

In [ ]:
import tweepy

# OAuth setup
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

### tweepy api 오브젝트 생성

In [ ]:
api = tweepy.API(auth)

In [ ]:
my_timeline = api.home_timeline()
my_timeline

# for tweet in my_timeline:
#     print(tweet.text)

### 특정 사용자의 타임라인 수집

In [ ]:
user = api.get_user("u_simin")
user

In [ ]:
user.id
user.description
user.created_at

In [ ]:
user = api.get_user(102026929)  # 만약 screen_name이 아닌 id를 알고 있다면

In [ ]:
user_timeline = api.user_timeline(user.id)
tweets = []
for tweet in user_timeline:
    tweets.append(tweet)

In [ ]:
tweets[1].text
tweets[1].user
tweets[1].user.name
tweets[1]._json

### 특정 유저의 친구 목록

In [ ]:
user = api.get_user("Yunaaaa")
user.screen_name
user.followers_count
user.friends_count

In [ ]:
for friend in user.friends():
    print(friend.screen_name)

(주의) user.followers()는 모두 665714 명이다. 트위터 API는 15분간 15개의 request call을 허용한다. 한번의 request 마다 20개의 followers를 가져온다면 모두 약 44381여개의 call이 필요하기 때문에 rate limit을 넘어서게 된다. 따라서 이 경우는 15개를 가져오고 잠시 쉬었다가 다시 가져오는 등의 방법을 사용한다. 처리 방법은 뒤에서 다루고 여기서는 call을 많이 사용하지 않는 friends의 리스트를 가져와 보았다.

https://dev.twitter.com/rest/public/rate-limiting

### Pagination and Cursor

트위터의 정보는 pagination 되어 있다. page를 넘겨가며 데이터를 수집하기 위해서는 cursor 를 사용한다. Cursor는 다음과 같이 사용한다.

In [ ]:
my_tweets = []
for tweet in tweepy.Cursor(api.user_timeline, id="u_simin").items(10):
    my_tweets.append(tweet)
    
my_tweets

In [ ]:
my_tweets[0].text
my_tweets[1].user.url

수집하고자 하는 트윗의 갯수를 지정할 수 있다. 이 경우 아이템의 숫자 (items()) 혹은 페이지의 숫자 (pages()) 를 지정한다.

```
# Only iterate through the first 200 statuses
for status in tweepy.Cursor(api.user_timeline).items(200):
    process_status(status)

# Only iterate through the first 3 pages
for page in tweepy.Cursor(api.user_timeline).pages(3):
    process_page(page)
```

### Handling the rate limit using cursors

위에서 언급한 것처럼, 많은 양의 트윗을 수집하고자 할 때, **```RateLimitError```** 에러가 발생한다. 이런 경우 페이지를 넘기기 전에 (next()) 에러를 체크하고, **```RateLimitError```** 에러가 발생하면 그 만큼 수집을 멈추었다가 재개하여야 한다. 에러 핸들링을 하지 않으면 프로그램이 멈추고 데이터 수집도 멈춘다.

```
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

for follower in limit_handled(tweepy.Cursor(api.followers).items()):
    process_follower(follower)
```

### Streaming Tweet Data

Streaming API는 rate limit의 제한이 없어 트윗을 수집할 때 유용하다. 그러나 모든 트윗을 제공하는 것은 아니고, 전체 트윗의 1%만을 random 으로 제공해 준다.

Streaming API를 사용하여 트윗을 수집하려면 다음과 같은 순서를 따른다.

* StreamListener 클래스를 상속받은 lister 클래스를 만든다.
* Stream 오브젝트를 생성한다.
* Stream 오브젝트에 Twitter API를 연결한다.

#### 1. StreamListener 클래스 생성

In [ ]:
import tweepy

class MyListener(tweepy.StreamListener):
    
    def on_status(self, data):
        print(data.text + "\n----")

#### 2. Stream 오브젝트 생성하고 Twitter API 연결 (OAuth 설정)

In [ ]:
twitter_stream = tweepy.Stream(auth, MyListener())

#### 3. Streaming 시작

In [ ]:
twitter_stream.filter(track=['trump', 'korea'])

#### 4. json 파일로 저장할 수 있도록 MyListener 클래스 수정

In [ ]:
class MyListener(tweepy.StreamListener):
    
    def on_data(self, data):
        try:
            with open('tweet_stream.json', 'a') as file:
                file.write(data)
                print(data)
                return True
        except BaseException as e:
            print("Error on_data: {}".format(str(e)))
        return True

In [ ]:
twitter_stream = tweepy.Stream(auth, MyListener())
twitter_stream.filter(track=['trump', 'korea'])

(참고1) ```on_status vs. on_data```

```on_status```는 데이터를 tweepy object 형태로 return한다. 따라서 data.text와 같은 notation이 가능하다. 하지만 ```on_data```는 데이터를 str로 return 한다. 우리는 데이터를 .json 파일로 저장할 것이기 때문에 tweepy object보다는 str로 저장하여야 한다. 따라서 두번째 수정된 코드에서는 ```on_data```를 사용하였다.

(참고2) ```on_status vs. on_data```

두개의 메소드는 StreamListener에 구현되어 있다. 클래스를 상속 받으면 부모 클래스에 구현된 메소드를 재정의(override)해서 사용할 수 있다.

### 터미널에서 프로그램 실행

jupyter notebook이 편리하긴 하지만 어떤 경우에는 터미널에서 프로그램을 실행하여야 한다. 스트리밍한 데이터를 저장하는 프로그램은 계속 데이터를 받아오기 때문에 jupyter notebook에서 실행하면 메모리 로드가 많아 작동불능 상태에 빠지기 쉽다. 따라서 이 경우 터미널에서 프로그램을 실행시킨다.

* 첫번째 스트리밍 코드는 twitter-stream.py 로 저장되어 있다.
* 수정된 두번째 스트리밍 코드는 twitter-stream-save.py로 저장되어 있다.

실행은 터미널에서 다음과 같이 한다.

* python twitter-stream-save.py

프로그램의 종료는 CTRL-C 를 누른다

### JSON 파일 불러오기

In [ ]:
import json

json_data = []
with open("tweet_stream.json") as file:
    data = file.readlines()
    for d in data:
        json_data.append(json.loads(d))

In [ ]:
i = 0
for tweet in json_data:
    if ("text" in tweet) and ("user" in tweet):
        print(str(i) + " - " + tweet["user"]["name"] + " :: " + tweet["text"])
        print("---")
        i = i + 1

## 2. OpenAPI 를 이용하여 데이터 크롤링

지금까지 살펴본 Twitter나 Facebook은 해당 서비스의 데이터 크롤링을 위한 파이썬 라이브러리를 이용하여 데이터를 수집할 수 있었다. 이들 라이브러리는 Twitter나 Facebook이 제공하는 OpenAPI에 맞게 개발되었다. 모든 서비스를 위한 라이브러리가 제공되는지는 않기 때문에 특정 서비스의 OpenAPI를 이용하려면 OpenAPI가 제공하는 방식에 맞게 프로그램을 설계하고 데이터를 수집한다.

대개 OpenAPI는 다음과 같은 형식의 URL을 이용하여 필요한 자료를 API 서버에 request 한다. (call)
* (포맷) http://web-address/api-name?param1=xxx&param2=xxx&param3=xxx
* (사례) http://api.openweathermap.org/data/2.5/weather?q=Seoul&units=metric&appid=yourkey

API서버가 call을 받으면 다음과 같이 JSON 포맷으로 결과를 반환한다. (response)
```
{
    "coord":{"lon":126.98,"lat":37.57},
    "weather":[
         {"id":701,"main":"Mist","description":"mist","icon":"50d"},
         {"id":500,"main":"Rain","description":"light rain","icon":"10d"},   
         {"id":721,"main":"Haze","description":"haze","icon":"50d"}
    ],
    "base":"stations",
    "main":{"temp":11.01,"pressure":1022,"humidity":43,"temp_min":10,"temp_max":12},
    "visibility":10000,
    "wind":{"speed":3.1,"deg":360},
    "clouds":{"all":90},
    "dt":1541917800,
    "sys":{"type":1,"id":7668,"message":0.0054,"country":"KR","sunrise":1541887637,"sunset":1541924664},
    "id":1835848,
    "name":"Seoul",
    "cod":200
}
```

이와 같이 API를 사용하기 위해서는 공개된 OpenAPI라 하더라도 개발자로 등록을 해야 한다.

개발자로 등록을 한 후에는 데이터 수집을 위해 제작할 application을 등록한다.

application을 등록하면 보통 app-key라는 것을 주는데, 이것은 일종의 아이디-패스워드이다. 즉, 누가 접속을 해서 데이터를 수집해가는 지를 서버에 알려주는 역할을 하며, 또한 서버 입장에서 데이터를 수집하는 앱의 트래픽을 콘트롤하기도 한다. (대개의 경우 call 숫자가 정해져 있다.)

본 예제에서는 openweathermap.org에서 제공하는 공개 데이터를 사용해 보고자 한다. 개발자 등록과 앱 등록은 슬라이드를 참고하자.

### 앱 기본 설정

app_key와 base_url 등 기본 설정을 한다.

In [ ]:
import urllib.request
import json
   
app_key  = ""
loc      = "Seoul"
base_url = "http://api.openweathermap.org/data/2.5/weather?q={}&units=metric&appid={}".format(loc, app_key)

### 현재 날씨

In [ ]:
print(base_url)

weather_data = ""
with urllib.request.urlopen(base_url) as response:
    weather_data = json.loads(response.read().decode("utf-8"))

#### 데이터 확인

In [ ]:
weather_data

### 반환되는 날씨 데이터의 정보는 다음을 참고해서 해석하자.
https://openweathermap.org/current
https://openweathermap.org/weather-conditions

(참고) `&lang=kr`를 URL에 넣어주면 한글로 데이터를 반환한다.

In [ ]:
humidity = weather_data["main"]["humidity"]
temp = weather_data["main"]["temp"]
cloud = weather_data["clouds"]["all"]
wind = weather_data["wind"]["speed"]
visibility = weather_data["visibility"]
print("습도: {}%".format(humidity))
print("현재기온: {}도".format(temp))
print("구름이 {}% 덮여있고 풍속은 {}m/s입니다.".format(cloud, wind))
print("현재 가시거리는 {}km입니다.".format(visibility/1000))

### UNIX Timestamp

유닉스 운영체계에서 자체적으로 시각을 나타내기 위해 사용하는 포맷을 Unix Timestamp 라고 한다. POSIX 나 Epoch 라고 부르기도 한다. 1970년 1월 1일 0시 0분 0초 (UTC: 협정 세계시) 부터 시작하고 음수로 표현되면 그 이전 시간을 의미한다. 32비트로 표현하며 1970년 1월 1일 부터 몇초가 지났는지를 표현한다.

* 예: 1541922737 seconds since Jan 01 1970. (UTC) 
* https://www.unixtimestamp.com/

UNIX Timestamp 를 변환하는 방법은 다음과 같다.

In [ ]:
from datetime import datetime

sunrise = weather_data["sys"]["sunrise"]
sunset = weather_data["sys"]["sunset"]

print(datetime.fromtimestamp(sunrise).strftime('%Y-%m-%d %H:%M:%S'))
print(datetime.fromtimestamp(sunset).strftime('%Y-%m-%d %H:%M:%S'))

print("오늘 일출시간은 {}이고, 일몰시간은 {}입니다.".format(
    datetime.fromtimestamp(sunrise).strftime('%H시 %M분'),
    datetime.fromtimestamp(sunset).strftime('%H시 %M분')))


### 미세먼지 정보

미세먼지 정보를 가져와보자.

미세먼지와 관련된 정보는 다음의 API를 활용할 예정이다.

* http://data.seoul.go.kr/dataList/datasetView.do?infId=OA-1204&srvType=A&serviceKind=1

데이터 수집을 위해서는 인증키(위의 App Key와 유사)을 받아야 하는데, 인증키 신청 버튼을 눌러 발급받도록 하자. 몇가지 form을 작성하면 키가 발급된다.

도큐멘테이션에 나와았는 API의 사용은 다음과 같다.

* http://openAPI.seoul.go.kr:8088/(인증키)/xml/ForecastWarningMinuteParticleOfDustService/1/1/ 

위의 URL의 (인증키) 부분에는 발급받은 인증키를 넣도록 하자.

현재 이 URL은 XML 형태로 데이터를 제공한다. xml을 json으로 바꿔서 json 형식으로 데이터를 받아보자.

In [ ]:
import urllib.request
import json
   
mykey  = ""
base_url = "http://openAPI.seoul.go.kr:8088/{}/json/ForecastWarningMinuteParticleOfDustService/1/1/ ".format(mykey)

fine_dust_data = ""
with urllib.request.urlopen(base_url) as response:
    fine_dust_data = json.loads(response.read().decode("utf-8"))

In [ ]:
fine_dust_data

In [ ]:
from datetime import datetime

time_observation = datetime.strptime(fine_dust_data["ForecastWarningMinuteParticleOfDustService"]["row"][0]["APPLC_DT"], "%Y%m%d%H%M")
dust_grade = fine_dust_data["ForecastWarningMinuteParticleOfDustService"]["row"][0]["CAISTEP"]
dust_grade

# · 0∼30 : 좋음 · 31∼80 : 보통 · 81∼150 : 나쁨 · 151 이상 : 매우 나쁨 


In [ ]:
warning = ""
if dust_grade == "매우 나쁨":
    warning = "외출을 자제하시는 것이 좋습니다."
elif dust_grade == "나쁨":
    warning = "외출시 꼭 마스크를 착용하세요."
elif dust_grade == "보통":
    warning = "마음껏 외출하세요."
else:
    warning = "너무 좋은 날씨네요. 마음껏 외출하세요 :)"
    
message = "{}년 {}월 {}일 {}시 현재, 미세먼지 수준은 {}입니다. ".format(time_observation.year, time_observation.month, time_observation.day, time_observation.hour, dust_grade)

print(message + warning)

## 4. Advanced Web Crawling

지난 시간에 다뤘던 것처럼, BeautifulSoup을 이용하여 HTML 내의 특정 요소를 찾아 데이터를 추출할 수 있다. 그러나 최근에 만들어지는 웹사이트의 경우, AJAX 를 사용하여 데이터를 불러오는 경우가 많아, 필요한 데이터를 추출하기 어려운 경우가 있다. 이런 경우, 브라우저에서 제공하는 developer tool 을 사용하여 데이터 소스를 찾아 직접 데이터를 다운로드 하기도 한다. (슬라이드 참조)

### 뉴스 사이트 (media daum)에서 기사의 본문을 추출

In [ ]:
from bs4 import BeautifulSoup
import urllib.request

url = "https://news.v.daum.net/v/20181111190005301"
doc = ""
with urllib.request.urlopen(url) as url:
    doc = url.read()

In [ ]:
soup = BeautifulSoup(doc, "html.parser")

In [ ]:
news_article = soup.find_all("div", class_="article_view")
news_article

In [ ]:
news_article[0].text

### 뉴스사이트에서 기사의 댓글을 추출

In [ ]:
comments = soup.find_all("ul", class_="list_comment")

In [ ]:
comments

위에서 기사의 본문을 추출한 것과 동일한 방법으로 기사의 댓글을 추출하고자 하였다. 브라우저의 inspector에서 보았을 때 해당 블락은 ```<ul class="list_comment">```로 정의되어 있었다. 그러나 실제로 해당 요소는 검색이 되지 않았다. 이는 페이지 소스에 이 요소가 포함되지 않았기 때문이다. 해당 요소는 page 가 모두 로딩된 후, javascript를 이용해서 페이지에 **삽입**된다. 따라서 이런 경우 데이터 수집이 불가능하다.

삽입되는 해당요소가 무엇인지를 찾기 위해서는 브라우저가 제공하는 개발자 도구에서 해당 요소를 삽입하는 코드를 찾아야 한다.
(슬라이드)

### JSON으로 댓글 추출하기

추출한 댓글 삽입 코드의 주소: https://comment.daum.net/apis/v1/posts/36628933/comments?parentId=0&offset=0&limit=3&sort=RECOMMEND

In [ ]:
import json

json_url = "https://comment.daum.net/apis/v1/posts/36628933/comments?parentId=0&offset=0&limit=3&sort=RECOMMEND"
with urllib.request.urlopen(json_url) as url:
    json_doc = url.read().decode("utf-8")
    json_data = json.loads(json_doc)
json_data

In [ ]:
for data in json_data:
    print(data['user']['displayName'])
    print(data['content'])
    print(data['likeCount'], data['dislikeCount'])
    print("--")

```parentId=0&offset=0&limit=3``` 의 패러미터 값을 조정하여 더 많은 데이터 수집 가능. 예를 들어 limit은 보여지는 댓글의 갯수를 얘기함.

limit을 100개로 고치고 프로그램을 다시 실행해 보자.

In [ ]:
json_url = "https://comment.daum.net/apis/v1/posts/36628933/comments?parentId=0&offset=0&limit=100&sort=RECOMMEND"
with urllib.request.urlopen(json_url) as url:
    json_doc = url.read().decode("utf-8")
    json_data = json.loads(json_doc)
    
for data in json_data:
    print(data['user']['displayName'])
    print(data['content'])
    print(data['likeCount'], data['dislikeCount'])
    print("--")